In [1]:
import os
import sys
script_dir = os.getcwd() # Get the current working directory
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))
from dotenv import load_dotenv
from src.paths import PARENT_DIR
load_dotenv(PARENT_DIR/'.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
HOPSWORKS_PROJECT_NAME = 'taxidemanpredictor'

In [2]:
from datetime import datetime
import pandas as pd 
from src.data import load_raw_data

# start = 2022
# end = datetime.now().year

# print(f'Downloading raw data from {start} to {end}')

# rides = pd.DataFrame()

# for year in range(start, end + 1):

#     rides_ = load_raw_data(year)

#     rides = pd.concat([rides, rides_])

In [3]:
# rides.shape

In [4]:
# from src.data import transform_raw_data_into_ts_data
# ts_data = transform_raw_data_into_ts_data(rides)

In [5]:
from src.paths import DATA_DIR
# ts_data.to_parquet(DATA_DIR / 'timeseries/ts_data_big.parquet')
ts_data = pd.read_parquet(DATA_DIR / 'timeseries/ts_data_big.parquet')

In [6]:
import hopsworks

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

/home/syelax/Desktop/real_world_ml/taxi-demand-prediction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-16 00:41:26,656 INFO: Initializing external client
2025-07-16 00:41:26,658 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-07-16 00:41:33,816 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240219


In [7]:
feature_store = project.get_feature_store()

In [9]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-Series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [10]:
feature_group.insert(ts_data, write_options={'wait_for_job':False})

Uploading Dataframe: 100.00% |██████████| Rows 7871064/7871064 | Elapsed Time: 07:49 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1240219/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)